<img src="files/logo.jpg"/>

## はじめに

このOptunaのチュートリアルは、Google Colaboratry で書かれたノートブックに沿って進めます。Google Colaboratry ではブラウザ上で Python プログラムを実行することができます。

試しに以下のコードを実行してみましょう。

In [ ]:
print('Hello Notebook!')  # shift + enter で実行できます

**このチュートリアルは、GPUを利用することで早く進めることができます。**

**メニューの ランタイム > ランタイムのタイプを変更 > ハードウェアアクセラレータ をGPUにしてください。**

Optunaは、次のようにインストールしてimportできます。

In [ ]:
!pip install -U setuptools
!pip install optuna

import optuna

## 1. Optuna を使って目的関数を最小化する

まずは簡単な問題として、Optuna を使って $(x - 2) ^ 2$ を最小化する $x$ を求めてみましょう。 
以下の 4 つのステップが必要となります。

1.   目的関数 (objective function) を定義する
2.   目的関数の内部で適当な変数を決める ***(suggest)***
3.   ***Study*** (実験) オブジェクトを作成する。
4.   ***Trial*** (試行) の回数を設定して最適化を開始する ***(optimize)***

目的関数を定義します。
`scipy.optimize`などの一般的な最適化ライブラリとは違い、$x$ の代わりに ***trial*** オブジェクトを引数にします。
これは Optuna で目的関数を書くときの決まりごとです。
***trial*** オブジェクトの ***suggest_float*** メソッド実行したタイミングで、次に試すべき $x$ の値が提案 ***(suggest)*** されます。
提案された $x$ を使って関数からの出力 $(x - 2) ^ 2$ を計算します。

In [ ]:
def objective(trial):
    x = trial.suggest_float("x", -100, 100)  # 区間　[-100, 100]　から適当な x を決める
    return (x - 2) ** 2

この目的関数を最小化する $x$ を求めることが Optuna の役目です。
以下のコードを実行してみましょう。
実験 ***(study)*** のオブジェクトを作成し、***optimize*** メソッドに目的関数と施行の回数を与えることで最小化実験が始まります。

In [ ]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

100 行のログが出力されているはずです。
Optuna が $(x - 2) ^ 2$ を 100 回実行し、そのつど異なる $x$ を試したことを意味しています。

実験結果を見てみましょう。
***study.best_value*** で 100 試行中の最小となる出力 $(x - 2) ^ 2$ の値が、***study.best_params*** でその時の入力 $x$ がわかります。

In [ ]:
print(f"目的関数の最小値: {study.best_value}")
print(f"出力が最小となる入力: {study.best_params}")

## 2. MNIST (手書き文字認識)

このチュートリアルでは例として、MNISTというデータセットと簡単なニューラルネットワークを使って手書き文字認識を行います。


MNISTには、次のような手書き文字が28x28の画像として訓練用60000枚 + 評価用10000枚収められています。

<img src="https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png" width="500"/>

(画像: https://ja.wikipedia.org/wiki/MNIST%E3%83%87%E3%83%BC%E3%82%BF%E3%83%99%E3%83%BC%E3%82%B9#/media/%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB:MnistExamples.png)

以下のプログラムを実行すると、自動的にデータセットがダウンロードされます。

In [ ]:
import torch
from torchvision.datasets import MNIST

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # あればGPUを使う

# データを読み込む (今回はデータが少ないので、全てメモリ上に入れる)
# 訓練用データセット
train = MNIST(root='./data', download=True, train=True)
train_X = train.data.to(device).to(torch.float32) # 60000x28x28
train_y = train.targets.to(device) # 60000

# 評価用データセット
validation = MNIST(root='./data', download=True, train=False)
validation_X = validation.data.to(device).to(torch.float32) # 10000x28x28
validation_y = validation.targets.to(device) # 10000

これらの画像を認識するために、今回は非常に単純な2層ニューラルネットワークを使って学習を行います。

<img src="https://upload.wikimedia.org/wikipedia/commons/c/c2/MultiLayerNeuralNetworkBigger_english.png" width="400">

(画像: https://simple.wikipedia.org/wiki/Deep_learning#/media/File:MultiLayerNeuralNetworkBigger_english.png)

今回は入力層(input layer)のサイズは画像サイズ28×28=784, 出力層(output layer)のサイズは0～9の文字に対応する10個です。中間層(hidden layer)のサイズと、中間層でかける活性化関数はうまく学習ができるように調整する必要があります。

深層学習フレームワークPyTorchを使うと、このモデルは次のように簡単に書けます。

In [ ]:
import torch.nn as nn

# モデルの定義
hidden_dim = 30            # 中間層の次元数
activation = nn.Sigmoid()  # 活性化関数

model = nn.Sequential(
    nn.Flatten(),                  # 二次元の画像を一次元に変換
    nn.Linear(28*28, hidden_dim),  # 入力層から中間層への線形変換
    activation,                    # 活性化関数
    nn.Linear(hidden_dim, 10),     # 中間層から出力層への線形変換
).to(device)


今はとりあえず中間層のサイズは30, 活性化関数はSigmoidを使っておきました。

データセットがあってモデルを立てたら、次にどのようにそのモデルのパラメタ(ハイパーパラメータではなく、モデル内の`nn.Linear`の重みなどのことです)を最適化するかを決めなければなりません。
今回は最も基本的な最適化アルゴリズムである確率的勾配降下法(Stochastic gradient descent; SGD)を使うことにします。

In [ ]:
import torch.optim as optim

# 最適化アルゴリズムの定義
lr = 1e-3  # 学習率
momentum = 0.9  # モーメンタム
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, nesterov=True)

モデルだけではなく最適化アルゴリズムにもハイパーパラメータはつきもので、学習率(`lr`)やモーメンタム(`momentum`)などをいい感じに調整する必要があります。

さて、今定義したモデルと最適化アルゴリズムを用いてモデルを訓練するコードは次のように書けます。以後何度もこのコードを使うので、モデルと最適化アルゴリズムを渡すとMNISTのデータで訓練と評価を行ってくれる関数を作ります。

In [ ]:
def fit_mnist_and_evaluate(model, optimizer):

    # 損失関数の定義
    loss_func = nn.CrossEntropyLoss() 

    # 学習 (今回はepoch数は20で固定とします。)
    epochs = 20
    for epoch in range(epochs):
        
        loss_sum = 0.0
        
        # batchをシャッフルする (今回はbatch sizeは600で固定とします。)
        batch_size = 600
        batch_idxs = torch.randperm(len(train_X), device=device).view(-1, batch_size)

        for i, batch in enumerate(batch_idxs):
            # 各batchについて最適化を回す

            optimizer.zero_grad() # 微分係数の初期化
            outputs = model(train_X[batch])          # 予測
            loss = loss_func(outputs, train_y[batch]) # 損失関数の計算 
            loss.backward()  # 微分の計算
            optimizer.step() # 最適化の1ステップの計算 

            loss_sum += loss.item()

        train_loss = loss_sum / (i + 1) # batchごとの損失関数の平均をとる

        # 評価
        with torch.no_grad():
            outputs = model(validation_X)
            _, predicted = torch.max(outputs.data, dim=1) # 最も予測値が高かったものをとる
            total = len(validation_X)
            correct = (predicted == validation_y).sum().item()
            validation_accuracy = correct / total

        print(f"Epoch {epoch + 1}: train_loss={train_loss:.3f}, validation_accuracy={validation_accuracy:.4f}")

    return validation_accuracy

`batch_size`も性能に影響を与えるため、場合によってはハイパーパラメータとして調整する必要があるかもしれません。今回はColabのGPUインスタンスで効率的に学習できる600で固定しておきました。

早速この関数を用いて学習してみましょう。

In [ ]:
fit_mnist_and_evaluate(model, optimizer)

学習がうまくいけば、validation accuracyが0.90～0.91前後になると思います。(実行するたびに変わります。)

この学習したモデルを使って認識した結果をいくつか表示してみましょう。

In [ ]:
import matplotlib.pyplot as plt

def visualize_model(model):

    fig, axs = plt.subplots(4, 5, figsize=(10, 11))
    axs = [ax for ax1 in axs for ax in ax1]
    start_idx = 500
    show_X = validation_X[start_idx:start_idx+len(axs)]
    show_y = validation_y[start_idx:start_idx+len(axs)]
    show_outputs = model(show_X)
    _, show_predicted = torch.max(show_outputs.data, dim=1)
    for i, ax in enumerate(axs):
        correct = show_y[i].item() == show_predicted[i].item()
        cmap = 'Greens' if correct else 'Reds'
        ax.imshow(show_X[i].cpu(), cmap=cmap)
        ax.set_title(f"Ground truth: {show_y[i].item()}\nPredicted: {show_predicted[i].item()}")
    
    return fig

fig = visualize_model(model)

正しく認識されたものは緑、間違って認識されたものは赤で表示しています。

この中にはいくつか誤認識されたものがあるかもしれません。では、次の節でモデルと最適化アルゴリズムのハイパーパラメタをoptunaで最適化して、認識精度を上げてみましょう。

## 3. Optuna で機械学習のハイパーパラメータを決める

今回調整するハイパーパラメータは、

* モデルの中間層のサイズ (`hidden_dim`)
* モデルの中間層の活性化関数 (`activation`)
* 最適化時の学習率 (`lr`)
* 最適化時のmomentum (`momentum`)

の4つです。これら全てを手動で調整するのは大変なので、今回はこの4つのハイパーパラメータをoptunaで自動的に調整して、モデルの精度を上げます。

(注: 実際に使われるより複雑なモデルでは、今回よりずっと多くのハイパーパラメータが含まれることがあります。)

Optunaでハイパーパラメータを調整するために、先ほどと同様に、`model`と`optimizer`を作って`fit_mnist_and_evaluate`を呼ぶプログラム全体を`objective`関数に包みます。

In [ ]:
def objective(trial):
        
    # モデルの定義
    hidden_dim = trial.suggest_int('hidden_dim', 10, 50)  # <-- 10から50までの間で探索
    activation_func = trial.suggest_categorical('activation_func', ['Sigmoid', 'Tanh', 'ReLU', 'ELU'])  # <-- この選択肢の中から探索
    activation_funcs = {
        'Sigmoid': nn.Sigmoid(),
        'Tanh': nn.Tanh(),
        'ReLU': nn.ReLU(),
        'ELU': nn.ELU(),
    }
    activation = activation_funcs[activation_func]

    model = nn.Sequential(
        nn.Flatten(),                  # 二次元の画像を一次元に変換
        nn.Linear(28*28, hidden_dim),  # 入力層から中間層への線形変換
        activation,                    # 活性化関数
        nn.Linear(hidden_dim, 10),     # 中間層から出力層への線形変換
    ).to(device)

    # 最適化アルゴリズムの定義
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True) # <-- 1e-5から1e-2までの間で、対数スケールで探索
    momentum = trial.suggest_float("momentum", 0.5, 1.0) # <-- 0.5から1.0までの間で探索
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, nesterov=True)

    validation_accuracy = fit_mnist_and_evaluate(model, optimizer)
    return validation_accuracy


コメントで`<--`と書いた4カ所と、最後に`return validation_accuracy`としているところが主な変更点です。

では、この目的関数をoptunaを用いて最適化しましょう。

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print(f"最良の精度: {study.best_value}")
print(f"最良のハイパーパラメータ: {study.best_params}")

うまくいけば0.94程度の精度が出ると思います。

得られたハイパーパラメータをはじめのの学習コードに入れて、もう一度学習を回してみてください。誤認識する画像が少なくなったでしょうか?

## 4. 高度なサンプリングアルゴリズム・枝刈りアルゴリズムを利用する

Optunaでは最先端のサンプリングアルゴリズム・枝刈りアルゴリズムを数多く利用して、最適化を行う事ができます。
その方法は非常にシンプルです。まずは、それぞれ**sampler**, **pruner**というオブジェクトを指定して、**study**を作成しましょう。

In [ ]:
sampler = optuna.samplers.TPESampler(multivariate=True)  # 多くの場合、multivariate=Trueを指定することで性能が少し上がる
pruner = optuna.pruners.HyperbandPruner()

study_with_pruner = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

**sampler**は、このようにstudyを作成する際に渡すだけで有効化されます。
一方で、**pruner**を有効にするためには、目的関数内で中間値を報告し、各ステップで枝刈りをするかどうか判断するコードを追加する必要があります。
上の分散並列最適化のセクションで用いた目的関数を、**pruner**が有効になるよう書き換えると以下のようになるでしょう。

In [ ]:
def fit_mnist_and_evaluate_with_report(trial, model, optimizer):

    # 損失関数の定義
    loss_func = nn.CrossEntropyLoss() 

    # 学習 (今回はepoch数は20で固定とします。)
    epochs = 20
    for epoch in range(epochs):
        
        loss_sum = 0.0
        
        # batchをシャッフルする (今回はbatch sizeは600で固定とします。)
        batch_size = 600
        batch_idxs = torch.randperm(len(train_X), device=device).view(-1, batch_size)

        for i, batch in enumerate(batch_idxs):
            # 各batchについて最適化を回す

            optimizer.zero_grad() # 微分係数の初期化
            outputs = model(train_X[batch])          # 予測
            loss = loss_func(outputs, train_y[batch]) # 損失関数の計算 
            loss.backward()  # 微分の計算
            optimizer.step() # 最適化の1ステップの計算 

            loss_sum += loss.item()

        train_loss = loss_sum / (i + 1) # batchごとの損失関数の平均をとる

        # 評価
        with torch.no_grad():
            outputs = model(validation_X)
            _, predicted = torch.max(outputs.data, dim=1) # 最も予測値が高かったものをとる
            total = len(validation_X)
            correct = (predicted == validation_y).sum().item()
            validation_accuracy = correct / total
        
        print(f"Epoch {epoch + 1}: train_loss={train_loss:.3f}, validation_accuracy={validation_accuracy:.4f}")

        # 中間値をOptunaに報告 <-- この3行が追加されている
        trial.report(validation_accuracy, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()

    return validation_accuracy

def objective_with_report(trial):
    
    # モデルの定義
    hidden_dim = trial.suggest_int('hidden_dim', 10, 50)  # 中間層の次元数
    activation_func = trial.suggest_categorical('activation_func', ['Sigmoid', 'Tanh', 'ReLU', 'ELU'])  # 活性化関数
    activation_funcs = {
        'Sigmoid': nn.Sigmoid(),
        'Tanh': nn.Tanh(),
        'ReLU': nn.ReLU(),
        'ELU': nn.ELU(),
    }
    activation = activation_funcs[activation_func]

    model = nn.Sequential(
        nn.Flatten(),                  # 二次元の画像を一次元に変換
        nn.Linear(28*28, hidden_dim),  # 入力層から中間層への線形変換
        activation,                    # 活性化関数
        nn.Linear(hidden_dim, 10),     # 中間層から出力層への線形変換
    ).to(device)

    # 最適化アルゴリズムの定義
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True) # <-- 1e-5から1e-2までの間で、対数スケールで探索
    momentum = trial.suggest_float("momentum", 0.5, 1.0) # <-- 0.5から1.0までの間で探索
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, nesterov=True)

    validation_accuracy = fit_mnist_and_evaluate_with_report(trial, model, optimizer)
    return validation_accuracy

主要な変更点は、`# 中間値をOptunaに報告`と書いた部分の3行です。

この目的関数を用いて**Study.optimize**を呼ぶと、枝刈りを考慮した最適化を行う事ができます。

In [ ]:
study_with_pruner.optimize(objective_with_report, n_trials=100)

print(f"最良の精度: {study_with_pruner.best_value}")
print(f"最良のハイパーパラメータ: {study_with_pruner.best_params}")

実行ログを見ると、いくつかのトライアルが最後まで実行されず途中で枝刈りされている事がわかります。
枝刈りによる高速化成功です！

...ちょっと待ってください。これで我々の最適化はリーズナブルに改善されたでしょうか？
いいえ。これだけだと、実行するはずだった100トライアルが早く終わっただけです。
枝刈りの目的は、与えられたリソースの元で、見込みの薄いトライアルを枝刈りすることで
最大限多くのハイパーパラメータ候補を試すことにあります。
したがって、枝刈りによって最適化を改善するためには**Study.optimizeのトライアル数を一定にしてはダメです。**
代わりに、**最適化全体で消費されたリソース量を一定にする必要があります。**

これを実現するための最も簡単な方法は、トライアル数ではなく時間を一定にすることです。


In [ ]:
study_with_pruner = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")
study_with_pruner.optimize(objective_with_report, timeout=60) # 60秒で終了

print(f"最良の精度: {study_with_pruner.best_value}")
print(f"最良のハイパーパラメータ: {study_with_pruner.best_params}")

こうすることで、与えられたリソース（60秒)の元で、見込みのないパラメータが枝刈りされて最大限多くのハイパーパラメータ候補が試されることになります。

最適化全体で消費されたリソース量を一定にする方法には、少しトリッキーですが以下のような方法も考えられるでしょう。
それは、最適化全体で実行されるステップ数を一定にすることです。

In [ ]:
study_with_pruner = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

# 枝刈りせずに学習した場合、100トライアル分のステップ数
N_TOTAL_STEPS = 100 * 20

def terminate_callback(study, trial):
    complete_steps = sum(
        len(t.intermediate_values) 
        for t in study.trials 
        if t.state == optuna.trial.TrialState.COMPLETE or t.state == optuna.trial.TrialState.PRUNED)
    if complete_steps >= N_TOTAL_STEPS:
        study.stop()

study_with_pruner.optimize(objective_with_report, callbacks=[terminate_callback])

print(f"最良の精度: {study_with_pruner.best_value}")
print(f"最良のハイパーパラメータ: {study_with_pruner.best_params}")

こうすることで、与えられたリソース（100 * 20ステップ数)の元で、見込みのないパラメータが枝刈りされて最大限多くのハイパーパラメータ候補が試されることになります。

さて、それではOptunaで利用可能な**sampler**と**pruner**について簡単に見ていきましょう。

Optunaでは、v3.1.0時点で以下のような**sampler**を利用する事ができます。
詳細は、各々のドキュメントを参照してください。
- [`optuna.samplers.TPESampler`](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.TPESampler.html)
  - ベイズ最適化のアルゴリズムの1つである[TPE (Tree-structured Parzen Estimator)](https://papers.nips.cc/paper/4443-algorithms-for-hyper-parameter-optimization.pdf)を実装しているsamplerです。
  - TPEはOptunaのデフォルトのsamplerです。
- [`optuna.samplers.CmaEsSampler`](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.CmaEsSampler.html)
  - 進化計算のアルゴリズムの1つである[CMA-ES (Covariance Matrix Adaptation - Evolution Strategy)](https://arxiv.org/abs/1604.00772) を実装しているsamplerです。
  - ハイパーパラメータの次元数が多い(10～20程度以上)場合や、trial数が多い(数百～数万程度)場合に強い手法です。
- [`optuna.integration.BoTorchSampler`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.integration.BoTorchSampler.html)
  - [GP (ガウス過程)](https://bayesoptbook.com/)を用いてベイズ最適化を行っているsamplerです。
  - 次元が少なく(～10程度)、trial数が少ない(～数百)場合に強い手法です。計算量がtrial数の3乗に比例するため、数百～千を超えるtrial数で用いることは現実的ではありません。
- [`optuna.samplers.NSGAIISampler`](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.NSGAIISampler.html)
  - 多目的最適化用の進化計算アルゴリズムの一つである[NSGA-II](https://ieeexplore.ieee.org/document/996017)を実装したsamplerです。
- [`optuna.samplers.GridSampler`](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.GridSampler.html)
  - グリッドサーチを行うsamplerです。
- [`optuna.samplers.BruteForceSampler`](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.BruteForceSampler.html)
  - 探索空間を全探索するsamplerです。`GridSampler`に似てますが、条件分岐を含むような探索空間にも対応しています。
- [`optuna.samplers.RandomSampler`](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.RandomSampler.html)
  - ランダムサーチを行うsamplerです。
- [`optuna.samplers.QMCSampler`](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.QMCSampler.html)
  - [準モンテカルロ法(quasi-Monte Carlo method)](https://scipy.github.io/devdocs/reference/stats.qmc.html)により、ランダムよりも満遍なく探索空間を埋め尽くすハイパーパラメータの列を用いて探索します。


また、Optunaでは、v3.1.0時点で以下のような**pruner**を利用する事ができます。
- [`optuna.pruners.NopPruner`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.pruners.NopPruner.html)
  - 何も枝刈りしない、というprunerです。
- [`optuna.pruners.PercentilePruner`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.pruners.PercentilePruner.html)
  - 各epochにおいて、最適化履歴の 下位$\alpha$%に入っていれば枝刈りする、というprunerです。
- [`optuna.pruners.MedianPruner`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.pruners.MedianPruner.html)
  - 各epochにおいて、最適化履歴の下位50%に入っていれば枝刈りする、というprunerです。
- [`optuna.pruners.SuccessiveHalvingPruner`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.pruners.SuccessiveHalvingPruner.html)
  - [Successive Halving](https://arxiv.org/abs/1810.05934)という有名な枝刈りアルゴリズムを実装しているprunerです。
  なお、Optunaでは実装の都合上、論文とは多少異なるアルゴリズムを実装しています。
- [`optuna.pruners.HyperbandPruner`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.pruners.HyperbandPruner.html)
  - [Hyperband](https://arxiv.org/abs/1603.06560)という最先端の枝刈りアルゴリズムを実装しているprunerです。
  なお、Optunaでは実装の都合上、論文とは多少異なるアルゴリズムを実装しています。
- [`optuna.pruners.ThresholdPruner`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.pruners.ThresholdPruner.html)
  - 各epochにおいて、与えられた閾値から外れた値の場合に枝刈りする、というprunerです。

以上のsampler, prunerをどういった組み合わせで用いれば良いのかは、タスクごとに異なります。

タスクごとに、どういった組み合わせでsamplerとprunerを選択すれば良いのかは難しい問題なので、我々Optuna開発者も多くのベンチマーク実験を行いながら知見を貯めています。
そういったベンチマーク実験の結果は[OptunaのWiki](https://github.com/optuna/optuna/wiki/Benchmarks-with-Kurobako)で公開されているので、samplerとprunerを選ぶ際は参考にしてみてください。

## 5. 可変の個数のハイパーパラメータを探索する

上ではかなり単純な二層ニューラルネットワークを使って手書き文字認識を行いましたが、もしかしたらより複雑なモデルを使うともっと精度を上げることができるかもしれません。ここでは中間層の数を可変にして、各層の次元数と活性化関数もOptunaに決めさせることを考えます。

調整するパラメタは以下の通りです。
- `n_hidden_layers`: 中間層の数。1～5の整数。
- 各中間層`i`に関して
  - `hidden_dim[i]`: 中間層`i`のサイズ
  - `activation_func[i]`: 中間層`i`の活性化関数
- 最適化時の学習率 (`lr`)
- 最適化時のmomentum (`momentum`)

この問題では、`n_hidden_layers`の値によって、必要なハイパーパラメータの数が変化することに注意してください。
このような場合でも、Optunaでは通常のPythonのコードを書くようにして扱うことができます。

In [ ]:
def objective_variable_depth(trial):
        
    # モデルの定義
    n_hidden_layers = trial.suggest_int('n_hidden_layers', 1, 5)  # 隠れ層の数
    activation_funcs = {
        'Sigmoid': nn.Sigmoid(),
        'Tanh': nn.Tanh(),
        'ReLU': nn.ReLU(),
        'ELU': nn.ELU(),
    }

    model = nn.Sequential()
    model.add_module('flatten', nn.Flatten()) # 二次元の画像を一次元に変換
    last_dim = 28 * 28 # 最初の入力層の次元数
    for i in range(n_hidden_layers):
        hidden_dim = trial.suggest_int(f'hidden_dim[{i}]', 10, 50) # 中間層の次元数
        activation_func = trial.suggest_categorical(f'activation_func[{i}]', ['Sigmoid', 'Tanh', 'ReLU', 'ELU']) # 中間層の活性化関数
        activation = activation_funcs[activation_func]

        model.add_module(f'linear[{i}]', nn.Linear(last_dim, hidden_dim)) # 線形変換
        model.add_module(f'activation[{i}]', activation) # 活性化関数
        last_dim = hidden_dim
    
    model.add_module(f'linear[{n_hidden_layers}]', nn.Linear(last_dim, 10)) # 出力層の線形変換

    model = model.to(device)

    # 最適化アルゴリズムの定義
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True) # 学習率
    momentum = trial.suggest_float("momentum", 0.5, 1.0) # モーメンタム
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, nesterov=True)

    validation_accuracy = fit_mnist_and_evaluate_with_report(trial, model, optimizer)
    return validation_accuracy


In [ ]:
study_variable_depth = optuna.create_study(
    direction="maximize", 
    sampler=sampler, 
    pruner=pruner,
)
study_variable_depth.optimize(objective_variable_depth, n_trials=100)

print(f"最良の精度: {study_variable_depth.best_value}")
print(f"最良のハイパーパラメータ: {study_variable_depth.best_params}")


恐らく3～4段くらい中間層を挟むことで、0.96～0.97程度まで性能が上がったのではないでしょうか?

このように通常のPythonコードを書くように探索空間が記述できることから、
Optunaの探索空間は"Pythonic search spaces"であると言われます。

## 6. `Study`の永続化と分散最適化

ここまでの例では、`optuna.create_study`で作られたStudyは実体がメモリ上に存在していたため、一つのプロセスからしかアクセスすることができず、またそのプロセスが終了するとStudyは失われてしまっていました。この節では、Optuna v3.1で新たに追加された`JournalStorage`を用いてStudyの実体をファイルに置く方法を紹介し、さらにそれを用いて複数プロセスで分散最適化を行います。

Studyの実体をファイルに置くには、`optuna.create_study`に次のような引数を渡します。

In [ ]:
distributed_study = optuna.create_study(
    direction="maximize",
    storage=optuna.storages.JournalStorage(optuna.storages.JournalFileStorage("./mnist_study.optuna")),  # 保存するファイルパスを指定
    study_name="distributed_study", # Study名を指定。既存のStudy名と同一のものを指定することで、同じStudyを参照できる。
    load_if_exists=True)


これまでとは違い、Studyに名前を与える必要があることに気をつけてください。同じファイルには複数のStudyが入ることが許され、それぞれのStudyはこの名前で区別されます。

`load_if_exists=True`と指定することで、既存のStudyと同じ名前が指定された場合に、新たにStudyが作られる代わりにそのStudyの実体と結びついたオブジェクトが返されます。例えばもう一度

In [ ]:
distributed_study2 = optuna.create_study(
    direction="maximize",
    storage=optuna.storages.JournalStorage(optuna.storages.JournalFileStorage("./mnist_study.optuna")),  # 保存するファイルパスを指定
    study_name="distributed_study", # Study名を指定。既存のStudy名と同一のものを指定することで、同じStudyを参照できる。
    load_if_exists=True)

と実行すると、`distributed_study`と`distributed_study2`は同じ実体を指し示すことになります。その証に、例えば

In [ ]:
distributed_study.optimize(objective_variable_depth, n_trials=1)
print(f"distributed_study.best_value = {distributed_study.best_value}")
print(f"distributed_study.best_params = {distributed_study.best_params}")

を実行した後、`distributed_study2`でも

In [ ]:
print(f"distributed_study2.best_value = {distributed_study2.best_value}")
print(f"distributed_study2.best_params = {distributed_study2.best_params}")

とすると、同じ値が表示されます。

これを用いて、分散最適化をしてみましょう。次のPythonスクリプトをダウンロードします。

In [ ]:
!wget https://raw.githubusercontent.com/pfnet-research/optuna-hands-on/master/ja/optuna_mnist_distributed_example.py

このスクリプトには、これまで書いたものと同じ`objective_variable_depth`の定義と、
```python
study_variable_depth = optuna.create_study(
    direction="maximize", 
    sampler=optuna.samplers.TPESampler(multivariate=True, constant_liar=True), 
    pruner=optuna.pruners.HyperbandPruner(),
    storage=optuna.storages.JournalStorage(optuna.storages.JournalFileStorage("./mnist_study.optuna")),  # 保存するファイルパスを指定
    study_name="distributed_study", # Study名を指定。既存のStudy名と同一のものを指定することで、同じStudyを参照できる。
    load_if_exists=True)
study_variable_depth.optimize(objective_variable_depth, n_trials=int(sys.argv[1]))
```
の2行が書かれています。`TPESampler`の`constant_liar`は分散最適化をするときに他のプロセスで実行中のハイパーパラメータと似たようなハイパーパラメータを提案してしまうことを防いで探索性能を上げるオプションです。(分散最適化をしないときは全く影響がありません。) `n_trials`はコマンドライン引数で指定できるようにしています。

分散最適化をするには、単にこのスクリプトを複数同時に走らせれば良いのです。(途中まで最適化を回した後、もう一度最初から実行させたい時は、`./mnist_study.optuna`を消してからもう一度コマンドを実行してください。)

In [ ]:
n_jobs = 4

# python optuna_mnist_distributed_example.py を並列で n_jobs 回呼ぶ。
!seq $n_jobs | xargs -P0 -n1 python optuna_mnist_distributed_example.py 25

これで4つのworkerで25trialずつ、合計100trialが実行されます。もし計算資源に余裕があれば、1プロセスで100trial実行するよりも早く終わるでしょう。

結果を見るのも、同じstudyを読み込むことでできます。

In [ ]:
distributed_study = optuna.load_study(
    storage=optuna.storages.JournalStorage(optuna.storages.JournalFileStorage("./mnist_study.optuna")),
    study_name="distributed_study")
# optuna.load_studyは、指定したstudyが存在しなければエラーになる。

print(f"最良の精度: {distributed_study.best_value}")
print(f"最良のハイパーパラメータ: {distributed_study.best_params}")


Optunaのこの`Storage`と呼ばれる枠組みは、非常に柔軟な分散最適化を可能にします。例えばこのファイルをNFS(ネットワークファイルシステム)上に置くことで、スパコン等の環境でもノードを超えた分散最適化が可能になります。また、メモリやファイルだけでなくRedis, あるいはMySQLやPostgreSQLなどのリレーショナルデータベース上にStudyの実体を置くこともできます。

## 7. 豊富な可視化機能を用いて最適化結果を分析する


高度なアルゴリズムの利用法、特に枝刈りの利用法について学び、最適化プロセスが改善されました。
また、必要があれば複数のワーカーで分散して並列処理をする事ができるようになりました。
最終的に達成された目的関数の評価値は**Study.best_value**, ハイパーパラメータの組は**Study.best_params**で取得できます。
最適化プロセスの改善や結果の分析は、これで十分でしょうか？

**いいえ。そんなことはありません。**

一度ハイパーパラメータ最適化を行うと、以下のような疑問が溢れてくることでしょう。
- トライアル数を決め打って最適化を行ったが、本当にこんなにたくさんのトライアルを実行する必要はあったのだろうか？
実はもっと早い段階で収束していたのではないだろうか？
逆にトライアル数は不十分だったりしないだろうか？

- 自分の目的関数において、最も適切なsamplerとprunerの組み合わせは何なのだろうか？
今使ったsamplerとprunerが本当に最適なのだろうか？

- 最適化の際に指定したハイパーパラメータの範囲は適切だろうか？
また、最終的に得たハイパーパラメータはどの程度信頼できるのだろうか？

- 最適化したハイパーパラメータの中で、目的関数に最もよく効いているパラメータは何なのだろうか？
逆に、効いていないパラメータは最適化せずとも良かったのではないだろうか？

Optunaは豊富な可視化機能を提供しており、ユーザはこれらの機能を利用して、
Optunaとインタラクティブに協同して、ハイパーパラメータ最適化のプロセスを改善していくことができます。
以下では、第3節(簡単な二層ニューラルネットワーク)で作った`study`を用いて、Optunaが提供する可視化機能の一部を紹介します。

まずは、`optuna.visualization.plot_optimization_history` 関数を紹介します。
この関数の利用法はとてもシンプルで、最適化した`study`を関数に渡すだけです。
この関数は、与えられたstudyで行われた最適化の様子を、横軸をトライアル数、縦軸を目的関数値として表示します。
表示されるのは枝刈りされずに最後まで完了したトライアルだけです。

In [ ]:
optuna.visualization.plot_optimization_history(study)

図が見にくかったら図の右上のボタンからズームしてみてください。

この図でもし早くから収束していてBest valueの線がそれ以上更新されてないようだと、実はこんなにtrialをたくさん回す必要がなかったということがわかります。逆に、まだまだ更新されるようでしたら、追加で`Study.optimize`を回すのがいいのかもしれません。

また、今回の目的関数にどのsamplerが適しているのか調べるために、複数のsamplerに対して`study`を作り最適化を行ってみましょう。

例えば、`QMCSampler`を使ってみます。`QMCSampler`は準モンテカルロ法(quasi-Monte Carlo method)と呼ばれる手法を用いて、「ランダムよりも一様な」数列を生成して満遍なく探索空間を試す、という手法です。グリッドサーチしたいけれど事前にどのくらい細かく分けるかは決めたくない、といった場合によく使える手法で、ランダムにとる`RandomSampler`よりも満遍なく試している分性能が高いことが多いです。

果たしてOptunaのデフォルトである`TPESampler`はこのような単純な手法と比べて性能が高いのか、調べてみましょう。

In [ ]:
# QMCSamplerはscipyが必要です
!pip install scipy
study_qmc = optuna.create_study(sampler=optuna.samplers.QMCSampler(), direction="maximize")
study_qmc.optimize(objective, n_trials=100)

print(f"最良の精度: {study_qmc.best_value}")
print(f"最良のハイパーパラメータ: {study_qmc.best_params}")

`TPESampler`での実行結果は第3節で実行してあるので、それと比較します。

In [ ]:
import plotly
fig1 = optuna.visualization.plot_optimization_history(study_qmc)
fig2 = optuna.visualization.plot_optimization_history(study)

fig1['data'][0]['name'] = 'Objective Value (QMC)'
fig1['data'][1]['name'] = 'Best Value (QMC)'
fig2['data'][0]['name'] = 'Objective Value (TPE)'
fig2['data'][1]['name'] = 'Best Value (TPE)'
fig = plotly.graph_objs.Figure(
    data=fig1['data'] + fig2['data'],
    layout=fig1['layout']
)
fig.show()

初めの10回程度の結果は、かなり運に左右されます。それ以降は、おそらく`TPESampler`の方が`QMCSampler`と比べて良い値をたくさんサンプルしており、より高い精度まで伸ばすことができているでしょう。

次に、`optuna.visualization.plot_contour`関数を紹介します。
この関数も、引数に`study`を与えるだけで動作します。
この関数は、全てのハイパーパラメータに対して、それらの任意の2個を縦軸横軸として目的関数値の等高線を表示します。（等高線を表示するハイパーパラメータの組を制限することもできます。詳細は[ドキュメント](https://optuna.readthedocs.io/en/stable/reference/visualization/generated/optuna.visualization.plot_contour.html#)を参照してください。）

In [ ]:
fig = optuna.visualization.plot_contour(study)
fig.update_layout(autosize=False, width=800, height=800) # 図が小さいのでサイズを調整

出力された等高線を見ると、`hidden_dim`は大きい方がよくて、`lr`は0.001以上だと非常に悪い結果になっていそう、ということが読み取れると思います。次に最適化するときは、例えば`lr`の探索範囲の上限を`1e-2`ではなく`1e-3`にしたり、`hidden_dim`の探索範囲の上限をより大きくしたりするとより早く良い解にたどり着くかもしれません。

最後に、`optuna.visualization.plot_param_importances`関数を紹介します。
この関数も最適化結果のstudyを渡すだけで利用することができます。
この関数は、最適化したハイパーパラメータに対して、目的関数の値に与えた影響の度合い（重要度）を計算して出力します。

上で計算した`study_new`をもとに、パラメータの重要度を計算してみましょう。

In [ ]:
optuna.visualization.plot_param_importances(study)

今回の目的関数に対しては、`lr`と`hidden_dim`が最も重要であり、`momentum`の寄与はほとんどないことがわかります。次に最適化するときは、例えば`momentum = 0.9`などと決め打ってやって、変数の数を減らすと早く最適化できるかもしれません。

## 8. Optuna Dashboard

実は、7節で行った可視化を全て自動で行い、Webインターフェイスで表示してくれるOptuna dashboardという便利なものがあります。この節ではOptuna dashboardの使い方を説明します。

まず、次のようにOptuna dashboardをインストールします。

In [ ]:
!pip install Cython==3.0.0a11
!pip install optuna-dashboard
!pip install optuna-fast-fanova gunicorn # Optuna dashboardの表示の高速化に役立つが、なくても構わない

Optuna dashboardで表示するためには、Studyが何らかのStorageに永続化されている必要があります。今回は第6節で作った`./mnist_study.optuna`をそのまま使います。

次のようなPythonコードを実行することで、Optuna dashboardのサーバーを立ち上げることができます。
```python
import optuna_dashboard, optuna
optuna_dashboard.run_server(optuna.storages.JournalStorage(optuna.storages.JournalFileStorage("./mnist_study.optuna")))
```
しかしこれをそのままJupyter notebook内でやってしまうと、notebookをブロックしてしまってサーバーを落とすまで他に何もできなくなるので、Jupyter notebookでは次のようにします。

In [ ]:
import time
import threading
from optuna_dashboard import wsgi
import optuna
from wsgiref.simple_server import make_server


port = 8081
storage = optuna.storages.JournalStorage(optuna.storages.JournalFileStorage("./mnist_study.optuna"))

app = wsgi(storage)
httpd = make_server("localhost", port, app)
thread = threading.Thread(target=httpd.serve_forever)
thread.start()

time.sleep(3) # サーバーが立ち上がるのを待つ。

try:
    from google.colab import output
    output.serve_kernel_port_as_window(port, path='/dashboard/')
except ModuleNotFoundError:
    # Google Colabではない環境
    print(f"https://localhost:{port}/dashboard/")

最も下に表示されたリンクをクリックしてください。Study名の一覧が表示され、先ほど設定した`distributed_study`を選択すると、様々な可視化を見ることができます。

さらに、このOptuna dashboardの画面は一定時間ごとに自動更新されます。試しに

In [ ]:
!python optuna_mnist_distributed_example.py 100

をもう一度呼んで、optuna dashboardがどう変化するか見てみてください。

次のようなコードを実行すると、Optuna dashboardのサーバーを止められます。

In [ ]:
# Dashboardの終了
httpd.shutdown()
httpd.server_close()
thread.join()

## おわりに

以上でOptunaのチュートリアルは終わりです。

今回はPyTorchを使った簡単なニューラルネットワークでMNISTを学習することを題材として、Optunaの基本的な使い方から少し高度な機能までざっと紹介しました。

今回は紹介できなかった優れたアルゴリズムや便利な可視化機能がまだまだ沢山あるので、ぜひ[ドキュメント](https://optuna.readthedocs.io/en/stable/index.html)を読んでみてくださいね。

[Optunaの解説本](https://www.amazon.co.jp/Optuna%E3%81%AB%E3%82%88%E3%82%8B%E3%83%96%E3%83%A9%E3%83%83%E3%82%AF%E3%83%9C%E3%83%83%E3%82%AF%E3%82%B9%E6%9C%80%E9%81%A9%E5%8C%96-%E4%BD%90%E9%87%8E-%E6%AD%A3%E5%A4%AA%E9%83%8E/dp/4274230104)も出ました！アルゴリズムの詳細などについても踏み込んで解説しているので、興味ある方はそちらも是非！

Optunaが気に入った方は、ぜひ[GitHubページ](https://github.com/optuna/optuna)でスターを押してください！

それでは、よいハイパラ最適化ライフを！